In [1]:
import win32com.client
import pandas as pd
import datetime

In [2]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

In [3]:
inbox = outlook.GetDefaultFolder(6) #6 is the inbox, can change it for other folders

In [4]:
messages = inbox.Items
messages.Sort("ReceivedTime")
message = messages.GetFirst()

#attach = message.Attachments
#print (body_content)
#print(datetime)
#print(sender)
#print(subject)

In [5]:
#create empty dataframe, fill it in as emails are read
df= pd.DataFrame(columns=['subject', 'sender', 'date', 'content of message'])

In [6]:
#first cast the datetime to string, so that it can be stripped of the timezone and used more easily
#The error correction can propably be improved
def get_date (date, i):
    date= str(date)
    try:
        date = datetime.datetime.strptime(date.rstrip("+00:00"), '%Y-%m-%d %H:%M:%S')
    except ValueError:
        date = datetime.datetime.strptime(date.rstrip("+00:00"), '%Y-%m-%d %H:%M')
    df.loc[i,'date'] =date
#read body firt, remove unwanted text, then feed it to the df
def get_content (body, i):
    body= body.replace("\r"," ") 
    body= body.replace("\n","") 
    body= body.replace("\t","") 
    df.loc[i,'content of message'] = body

In [7]:
for i in range(0,200):
    try:
        df.loc[i,'subject']=message.Subject
        df.loc[i,'sender']=message.Sender
        get_date(message.CreationTime, i)      
        get_content(message.body, i)
        
    except AttributeError as e:
        if e == 'GetNext.Subject':
            df.loc[i,'subject']= None
            df.loc[i,'sender'] = message.Sender
            get_date(message.CreationTime, i)
            get_content(message.body, i)
        elif e == 'GetNext.Sender':
            df.loc[i,'sender'] = None
            get_date(message.CreationTime, i)
            get_content(message.body, i)
        elif e=='GetNext.message.body':
            df.loc[i,'content of message'] = None
           
            
    message= messages.GetNext()



In [8]:
display(df)

,subject,sender,date,content of message
0,Databases (2017-100-KEN2110): Group 1 and 2,Dr Mena Habib Habib - m.habib@maastrichtuniver...,2017-09-18 09:45:55,Dear DB Students Since tomorrow you should fol...
1,ISAP Intervision Reminder - this Wednesday!,Info-isap (SSC),2017-09-18 10:59:33,"Dear Ambassador, This is a reminder to re..."
2,contract Vodafone,"Narinx, Ellen (DKE)",2017-09-18 14:26:58,"Hi Isabel, Last week we spoke about the fu..."
3,Probability and Statistics (2017-100-KEN2130):...,Georgios Stamoulis - georgios.stamoulis@maastr...,2017-09-19 11:32:09,"Hi all, this is a summary of yesterday's Lect..."
4,KE@Work internship projectform,Kework (DKE),2017-09-19 14:03:38,"Dear all, Please find attached the interns..."
5,UM News,Maastricht University,2017-09-19 15:13:24,19-09-2017 View this <http://mailing.maastric...
6,temporary move of teaching activities Tapijn Z...,"Custers, Jet (DKE)",2017-09-20 13:27:04,"Dear student, I would like to inform you ..."
7,Probability and Statistics (2017-100-KEN2130):...,Georgios Stamoulis - georgios.stamoulis@maastr...,2017-09-20 13:49:16,"Hi All, find in attachment the notes that cor..."
8,"“Move more, feel better” - Active Campus Europe",Incognito (DKE),2017-09-20 15:29:02,"Dear DKE student, UM Sport has asked us t..."
9,Probability and Statistics (2017-100-KEN2130):...,Georgios Stamoulis - georgios.stamoulis@maastr...,2017-09-20 17:09:11,"Dear all, In case you are wondering, in the Q..."


In [9]:
#TODO: read all emails and write it to a csv file
df.to_csv('.\dataset\Isabel_Email.csv')

In [ ]:
import smtplib
import time
import imaplib
import email

In [ ]:
ORG_EMAIL   = "@student.maastrichtuniversity.nl"
EMAIL  = "i.pereirafernandez"+ ORG_EMAIL 
PWD    = "Marto1408"
SMTP_SERVER = "outlook.office365.com"
SMTP_PORT   = 993

#double login, second step (through UM portal, requires student Number, not email)

In [ ]:
mail = imaplib.IMAP4_SSL(SMTP_SERVER)

In [ ]:
mail.login(EMAIL,PWD)